# Preprocessing for Regression and Classification
The features we want to use for our regression and classification algorithms are in different files and in different formats. To generate a single dataframe containing all the information needed, the following is done: 
1. Extract and prepare information from movies.csv
2. Merge with links.csv to get the needed IDs
3. Extract and prepare information from ratings.csv
4. Extract and prepare information from credits.csv 
5. Hot Encoding of further attributes
6. Integrate Cluster ID
7. Export

In [1]:
# change used width of browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### Import libraries

In [2]:
import pandas as pd
import re
import seaborn as sb
import matplotlib.pyplot as plt
import datetime
import json
import warnings
import numpy as np
from sklearn.preprocessing import LabelEncoder
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1. Extract and prepare information from movies.csv

### 1.1. Load movies.csv

In [2]:
df_movies = pd.read_csv("the-movies-dataset/movies_metadata.csv")
print("Length at import: " + str(len(df_movies)))

# filter movies on Status == 'Released' and drop status column
df_movies = df_movies[df_movies["status"] == 'Released']

print("Length with only released movies: " + str(len(df_movies)))

# copy of original data
df_movies_raw = df_movies.copy()

Length at import: 45463
Length with only released movies: 45017


### 1.2. Analyze Genres before Hot Encoding
#### How is the distribution of genres?

In [3]:
genres_stat = {}
for index, row in df_movies.iterrows():
    genres = json.loads(re.sub("'", '"',row.genres))
    for genre in genres: # turn string to json and loop over genres
        genres_stat.update({genre.get("name") : 0})

#Get the movie count per genre
movies = {}
for index, row in df_movies.iterrows():
    tmp_genres = []  
    genres = json.loads(re.sub("'", '"',row.genres))
    for genre in genres:
        tmp_genres.append(genre.get("name"))
        new_value = genres_stat.get(genre.get("name")) + 1
        genres_stat.update({genre.get("name") : new_value})
    movies.update({row.id:{'title': row.title, 'genres': tmp_genres}})

for key, value in sorted(genres_stat.items(), key=lambda item: item[1], reverse=True):
    print("%s: %s" % (key, value))

Drama: 20079
Comedy: 13079
Thriller: 7572
Romance: 6668
Action: 6548
Horror: 4637
Crime: 4286
Documentary: 3874
Adventure: 3474
Science Fiction: 3020
Family: 2743
Mystery: 2456
Fantasy: 2294
Animation: 1917
Foreign: 1588
Music: 1588
History: 1391
War: 1318
Western: 1038
TV Movie: 760


### 1.3. Genres hot encoden

As there are not so many genres, we decided to hot encode all of them.

In [4]:
genre_set = {""}
for index, row in df_movies.iterrows():
    genres = json.loads(re.sub("'", '"',row.genres))
    for genre in genres: # turn string to json and loop over genres
        genre_set.add(genre.get("name"))       
genre_set.remove("")

for g in genre_set:
    df_movies[g] = 0

# 2. add genres as columns to dataframe, default value = 0
df_movie_genres = pd.DataFrame(columns=genre_set)
for index, row in df_movies.iterrows():
    temp = pd.Series(index=genre_set)
    tmp_genres = []  
    genres = json.loads(re.sub("'", '"',row.genres))
    for genre in genres:
        if genre.get("name") in list(genre_set):
            tmp_genres.append(genre.get("name"))
    for genre in tmp_genres:
        temp[genre] = 1
    df_movie_genres = df_movie_genres.append(temp, ignore_index=True)

# 3. add one hot encoded genres to df_movies        
df_movies[list(genre_set)] = df_movie_genres

# 4. turn NaN values to zeros
df_movies = df_movies.fillna(0)

### 1.4. Encoding of belongs_to_collection, adult and homepage

We want to convert binary attributes into ones and zeros to have it standardized.

In [5]:
df_movies['part_of_collection'] = ""
df_movies['+18'] = ""
df_movies['hasHomepage'] = ""

for index, row in df_movies.iterrows():
    if row.belongs_to_collection == 0:
        df_movies.set_value(index, 'part_of_collection', 0)
    else:
        df_movies.set_value(index, 'part_of_collection', 1)
    
    if row.adult == "True":
        df_movies.set_value(index, '18+', 1)
    else:
        df_movies.set_value(index, '18+', 0)   
    
    pattern = 'www.'
    tmp = re.search(pattern, str(row.homepage))
    if tmp != None:
        df_movies.set_value(index, 'hasHomepage', 1)
    else:
        df_movies.set_value(index, 'hasHomepage', 0)

### 1.5. Extract attributes from Json values

Some of the values are in JSON format. In this step we extract the information using regular expressions because reading the values with the Json module did not work properly.

In [6]:
import json
pattern1 = "{'name': '([a-zA-Z ]*)'"
pattern2 = "'name': '([a-zA-Z ]*)'"

df_movies['productionCompanies'] = ""
df_movies['productionCountries'] = ""
df_movies['spokenLanguages'] = ""
for index, row in df_movies.iterrows():
    companies = []
    countries = []
    lang = []
    
    text1 = row.production_companies
    liste1 = text1.split("}")
    for i in liste1:
        tmp_name = re.search(pattern1, i)
        if tmp_name != None:
            companies.append(tmp_name.group(1))
    df_movies.set_value(index, 'productionCompanies', ','.join(map(str, companies)))
    
    text2 = row.production_countries
    liste2 = text2.split("}")
    for i in liste2:
        tmp_name = re.search(pattern2, i)
        if tmp_name != None:
            countries.append(tmp_name.group(1))
    df_movies.set_value(index, 'productionCountries', ','.join(map(str, countries)))
    
    text3 = row.spoken_languages
    liste3 = text3.split("}")
    for i in liste3:
        tmp_name = re.search(pattern2, i)
        if tmp_name != None:
            lang.append(tmp_name.group(1))
    df_movies.set_value(index, 'spokenLanguages', ','.join(map(str, lang)))

### 1.6. Adjust ratingID before merge

We need the IMDB ID to match the records in links.csv. The problem was that the ID in the movies.csv has "tt" at the beginning. We remove those so that a match is possible.

In [7]:
df_ids = pd.DataFrame(columns=['ratingID'])
df_ids.ratingID = df_ids.ratingID.astype('int64')
pattern = "^tt(\d*)"
for index, row in df_movies.iterrows():
    temp = pd.Series(['ratingID'])
    newValue = str(row.imdb_id).replace("t","",-1)
    temp['ratingID'] = int(newValue)
    df_ids = df_ids.append(temp, ignore_index=True)
df_movies['ratingID'] = df_ids['ratingID']

## 2. Merge with links.csv to get the needed IDs

links.csv represents the "bridge" to cast.csv containing the one common ID - imdbID

### 2.1. Load Data

In [8]:
df_links = pd.read_csv("the-movies-dataset/links.csv")
print("Length at import (Links): " + str(len(df_links)))
df_links.head(3)

Length at import (Links): 45843


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


### 2.2. Merge Data

In [9]:
df_tmp = pd.merge(left=df_movies,right=df_links, left_on='ratingID', right_on='imdbId', how='inner')
print("Length after Merge (Movies + Links): " + str(len(df_tmp)))
df_tmp.head(3)

Length after Merge (Movies + Links): 44508


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,+18,hasHomepage,18+,productionCompanies,productionCountries,spokenLanguages,ratingID,movieId,imdbId,tmdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,,0,0.0,Pixar Animation Studios,United States of America,English,114709.0,1,114709,862.0
1,False,0,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",0,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,,0,0.0,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,English,113497.0,2,113497,8844.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,,0,0.0,Lancaster Gate,United States of America,English,113228.0,3,113228,15602.0


## 3. Extract and prepare information from ratings.csv

We need rating.csv to calculate the average rating for every movie. This will be our target.

### 3.1. Load Data

In [10]:
df_ratings = pd.read_csv("the-movies-dataset/ratings.csv")
print("Length at import (Ratings): " + str(len(df_ratings)))
df_ratings.head(3)

Length at import (Ratings): 26024289


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523


### 3.2. Generate a Dataframe with movieId and the average rating

In [11]:
# calculate mean for every movie
df_ratings_grouped = df_ratings.groupby('movieId')[['rating']].mean()

### 3.3. Merge Data

In [12]:
df_joined = pd.merge(left=df_tmp,right=df_ratings_grouped, left_on='movieId', right_on='movieId', how='inner')
print("Length after merge (Movies + Links + Ratings): " + str(len(df_joined)))
df_joined.head(3)

Length after merge (Movies + Links + Ratings): 43803


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,hasHomepage,18+,productionCompanies,productionCountries,spokenLanguages,ratingID,movieId,imdbId,tmdbId,rating
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0.0,Pixar Animation Studios,United States of America,English,114709.0,1,114709,862.0,3.888157
1,False,0,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",0,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,0,0.0,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,English,113497.0,2,113497,8844.0,3.236953
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0,0.0,Lancaster Gate,United States of America,English,113228.0,3,113228,15602.0,3.175550


### 3.4. Remove features, which are not needed
Some features are not useful like original_title and some features can't be used according to our use case like revenue or vote count.

In [13]:
# clean up dataframe - remove features, which are not needed
features_to_remove = ['genres','original_title','overview','popularity','poster_path','release_date','revenue','status','tagline','title','video','vote_average','vote_count','belongs_to_collection','adult','homepage','imdb_id','production_companies','production_countries','spoken_languages','tmdbId','ratingID']

for i in features_to_remove:
    if i in df_joined.columns:
        df_joined = df_joined.drop(columns=i)
df_joined.head(5)

,budget,id,original_language,runtime,Documentary,Foreign,Action,Horror,War,Romance,...,part_of_collection,+18,hasHomepage,18+,productionCompanies,productionCountries,spokenLanguages,movieId,imdbId,rating
0,30000000,862,en,81.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,,0,0.0,Pixar Animation Studios,United States of America,English,1,114709,3.888157
1,65000000,8844,en,104.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,,0,0.0,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,English,2,113497,3.236953
2,0,15602,en,101.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1,,0,0.0,Lancaster Gate,United States of America,English,3,113228,3.175550
3,16000000,31357,en,127.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,,0,0.0,Twentieth Century Fox Film Corporation,United States of America,English,4,114885,2.875713
4,0,11862,en,106.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,,0,0.0,"Sandollar Productions,Touchstone Pictures",United States of America,English,5,113041,3.079565


### 3.5. Export Data for Clustering

In [14]:
# Export for Clustering
df_joined.to_csv("clusterPreprocessing.csv", index=False)

## 4. Extract and prepare information from credits.csv

credits.csv contains actors and directors for every movie. These are also saved as JSON and have to be extracted using regular expressions.

### 4.1. Load Data

In [15]:
# cast und crew integrieren
df_credits = pd.read_csv("the-movies-dataset/credits.csv")
print("Length at import (Credits): " + str(len(df_credits)))
df_credits.head(5)

Length at import (Credits): 45476


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


### 4.2. Extract Director

In [16]:
pattern = '\"Director\", \"name\": \"([a-zA-Z ]*)\", \"'
directors = set()
df_credits['director'] = ""
for index, row in df_credits.iterrows():
    text = row.crew
    text = text.replace('\\','',-1)
    text = text.replace("'",'"',-1)
    a = re.search(pattern, text)
    if a != None:
        directors.add(a.group(1))
        df_credits.set_value(index, 'director', a.group(1))
print(len(directors))
#print(directors)
df_credits.head(3)

14945


,cast,crew,id,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch


### 4.3. Extract actors

In [17]:
pattern = '\"name\": \"(.*)\", '
df_credits['actors'] = ""
for index, row in df_credits.iterrows():
    actors = []
    text = row.cast
    text = text.replace('\\','',-1)
    text = text.replace("'",'"',-1)
    liste = text.split("cast_id")
    for i in liste:
        tmp_name = re.search(pattern, i)
        if tmp_name != None:
            actors.append(tmp_name.group(1))
    df_credits.set_value(index, 'actors', ','.join(map(str, actors)))
df_credits.head(3)

,cast,crew,id,director,actors
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."


### 4.4. Remove features which are not needed

In [18]:
df_credits = df_credits.drop(columns=['cast','crew'])
df_joined = pd.merge(left=df_joined,right=df_credits, left_on='id', right_on='id', how='inner')
print("Length after merge (Movies + Ratings + Credits): " + str(len(df_joined)))
df_joined.head(3)

Length after merge (Movies + Ratings + Credits): 43872


,budget,id,original_language,runtime,Documentary,Foreign,Action,Horror,War,Romance,...,hasHomepage,18+,productionCompanies,productionCountries,spokenLanguages,movieId,imdbId,rating,director,actors
0,30000000,862,en,81.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,Pixar Animation Studios,United States of America,English,1,114709,3.888157,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,65000000,8844,en,104.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,English,2,113497,3.236953,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,0,15602,en,101.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0.0,Lancaster Gate,United States of America,English,3,113228,3.175550,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."


## 5. Hot Encoding of further attributes
Since not all list based attributes can not be hot encoded (high number of columns, performance), we decided to take the most common values.

### 5.1. Overview

In [19]:
print(str(len(df_joined.original_language.unique())) + " original languages")
print(str(len(df_joined.spokenLanguages.unique())) + " spoken languages")
print(str(len(df_joined.productionCompanies.unique())) + " production companies")
print(str(len(df_joined.productionCountries.unique())) + " production countries")
print("Max Budget: " + str(df_joined.budget.max()))
print("Min Budget: " + str(df_joined.budget.min()))

90 original languages
410 spoken languages
18396 production companies
2338 production countries
Max Budget: 380000000
Min Budget: 0


### 5.2. Finding most important production companies

In [20]:
dict_companies = {}
for index, row in df_joined.iterrows():
    liste_companies = row.productionCompanies.split(",")
    for i in liste_companies:
        if i in dict_companies:
            dict_companies[i] = dict_companies[i] + 1
        else:
            dict_companies[i] = 1
del dict_companies['']

print("Overall Number of Production Companies: ", len(dict_companies))
print("Number of different occurances: ", len(list(set(dict_companies.values()))))

print("\n### Top Companies ###")
for key, value in sorted(dict_companies.items(), key=lambda item: item[1], reverse=True):
    if value > 100:
        print("%s: %s" % (key, value))
    else:
        break

companies = []
for k, v in dict_companies.items():
    if v > 100: companies.append(k)

print("Number of most common companies: ", len(companies))

Overall Number of Production Companies:  17703
Number of different occurances:  103

### Top Companies ###
Paramount Pictures: 985
Twentieth Century Fox Film Corporation: 824
Universal Pictures: 814
Columbia Pictures Corporation: 440
Columbia Pictures: 424
RKO Radio Pictures: 287
United Artists: 275
New Line Cinema: 274
Walt Disney Pictures: 260
Touchstone Pictures: 223
TriStar Pictures: 197
Miramax Films: 183
Mosfilm: 175
Toho Company: 144
BBC Films: 128
Gaumont: 119
StudioCanal: 118
Relativity Media: 118
Walt Disney Productions: 115
Orion Pictures: 110
Village Roadshow Pictures: 106
Regency Enterprises: 105
Number of most common companies:  22


### 5.3. Finding most important actors (using number of occurences)

In [21]:
dict_actors = {}
for index, row in df_joined.iterrows():
    liste_actors = row.actors.split(",")
    for i in liste_actors:
        if i in dict_actors:
            dict_actors[i] = dict_actors[i] + 1
        else:
            dict_actors[i] = 1
del dict_actors['']
del dict_actors[' Jr.']
          
print("Overall Number of Actors: ", len(dict_actors))
print("Number of different occurances: ", len(list(set(dict_actors.values()))))

print("\n### Top Actors ###")
for key, value in sorted(dict_actors.items(), key=lambda item: item[1], reverse=True):
    if value > 100:
        print("%s: %s" % (key, value))
    else:
        break

actors = []
for k, v in dict_actors.items():
    if v > 100: actors.append(k)

print("\nNumber of most common actors: ", len(actors))

Overall Number of Actors:  198328
Number of different occurances:  107

### Top Actors ###
Bess Flowers: 238
Christopher Lee: 140
John Wayne: 124
Samuel L. Jackson: 121
Michael Caine: 110
Donald Sutherland: 109
Jackie Chan: 107
Gérard Depardieu: 107
John Carradine: 106
Robert De Niro: 104
Frank Welker: 103

Number of most common actors:  11


### 5.4. Definition of functions for (Hot) Encoding

In [22]:
def originalLanguageEncoding(df):
    lang_set = {""}
    for index, row in df.iterrows():
        lang_set.add("orig_" + str(row.original_language))      
    lang_set.remove("")

    for g in lang_set:
        df[g] = 0

    df_movie_lang = pd.DataFrame(columns=lang_set)
    for index, row in df.iterrows():
        if index % 5000 == 0:
            print((index/len(df))*100)
        temp = pd.Series(index=lang_set)
        tmp_lang = "orig_" + str(row.original_language)
        temp[tmp_lang] = 1
        df_movie_lang = df_movie_lang.append(temp, ignore_index=True)

    # 3. add one hot encoded genres to df_movies
    df[list(lang_set)] = df_movie_lang

    # 4. turn NaN values to zeros
    df = df_joined.fillna(0)

In [23]:
def productionCompaniesEncoding(df):
    # companies contains only the important ones
    pland_set = {""}
    for index, row in df.iterrows():
        pcs = str(row.productionCompanies).split(",")
        for pc in pcs:
            if pc in companies:
                pland_set.add("pcomp_" + str(pc))       
    pland_set.remove("")

    for g in pland_set:
        df[g] = 0

    # 2. add genres as columns to dataframe, default value = 0
    df_movie_pc = pd.DataFrame(columns=pland_set)
    for index, row in df.iterrows():
        if index % 5000 == 0:
            print((index/len(df))*100)
        temp = pd.Series(index=pland_set)
        tmp_pcs = []
        pcs = str(row.productionCompanies).split(",")
        for pc in pcs:
            if pc in list(pland_set):
                tmp_pcs.append("pcomp_" + str(pc))
        for pc in tmp_pcs:
            temp[pc] = 1
        df_movie_pc = df_movie_pc.append(temp, ignore_index=True)

    # 3. add one hot encoded genres to df_movies        
    df[list(pland_set)] = df_movie_pc

    # 4. turn NaN values to zeros
    df = df.fillna(0)
    
    return df

In [24]:
def actorsEncoding(df):
    # actors contains only the important ones
    actors_set = {""}
    for index, row in df.iterrows():
        actors_tmp = str(row.actors).split(",")
        for actor in actors_tmp:
            if actor in actors:
                actors_set.add("actor_" + str(actor))       
    actors_set.remove("")

    for actor in actors_set:
        df[actor] = 0

    # 2. add genres as columns to dataframe, default value = 0
    df_movie_actors = pd.DataFrame(columns=actors_set)
    for index, row in df.iterrows():
        if index % 5000 == 0:
            print((index/len(df))*100)
        temp = pd.Series(index=actors_set)
        tmp_actors = []
        actors_list = str(row.actors).split(",")
        for act in actors_list:
            if act in list(actors_set):
                tmp = "actor_" + str(act)
                temp[tmp] = 1
        df_movie_actors = df_movie_actors.append(temp, ignore_index=True)

    # 3. add one hot encoded genres to df_movies        
    df[list(actors_set)] = df_movie_actors

    # 4. turn NaN values to zeros
    df = df.fillna(0)
    
    return df

In [25]:
def directorEncoding(df):
    lab_enc = LabelEncoder()
    df['director'] = lab_enc.fit_transform(df['director'])
    return df

### 5.5. Hot Encoding for Production Companies, Actors and Original Languages

In [26]:
#print("Start Hot Encoding for original languages...")
#df_joined = originalLanguageEncoding(df_joined) --> not useful
print("Start Encoding for Director...")
df_joined = directorEncoding(df_joined)
print("Start Hot Encoding for Companies...")
df_joined = productionCompaniesEncoding(df_joined)
df_joined.head(5)

Start Encoding for Director...
Start Hot Encoding for Companies...
0.0
11.396790663749089
22.793581327498178
34.19037199124727
45.587162654996355
56.98395331874544
68.38074398249454
79.77753464624362
91.17432530999271


,budget,id,original_language,runtime,Documentary,Foreign,Action,Horror,War,Romance,...,pcomp_Mosfilm,pcomp_RKO Radio Pictures,pcomp_Miramax Films,pcomp_TriStar Pictures,pcomp_Columbia Pictures,pcomp_Toho Company,pcomp_Relativity Media,pcomp_Walt Disney Pictures,pcomp_BBC Films,pcomp_United Artists
0,30000000,862,en,81.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,65000000,8844,en,104.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0,15602,en,101.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,16000000,31357,en,127.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,0,11862,en,106.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
print("Start Hot Encoding for Actors...")
df_joined = actorsEncoding(df_joined)
df_joined.head(5)

Start Hot Encoding for Actors...
0.0
11.396790663749089
22.793581327498178
34.19037199124727
45.587162654996355
56.98395331874544
68.38074398249454
79.77753464624362
91.17432530999271


,budget,id,original_language,runtime,Documentary,Foreign,Action,Horror,War,Romance,...,actor_John Wayne,actor_Donald Sutherland,actor_Robert De Niro,actor_Samuel L. Jackson,actor_Jackie Chan,actor_Michael Caine,actor_Christopher Lee,actor_Frank Welker,actor_John Carradine,actor_Gérard Depardieu
0,30000000,862,en,81.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,65000000,8844,en,104.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0,15602,en,101.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,16000000,31357,en,127.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,0,11862,en,106.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


### 5.6. Export results / import results

In [30]:
df_joined.to_csv("regressionPreprocessing_backup.csv", index=False)

In [37]:
df_joined = pd.read_csv("regressionPreprocessing_backup.csv")

### 5.7. Normalization of Budget and Runtime

In [ ]:
def normalization():
    print("## Budget ##")
    print("Max: " + str(df_joined['budget'].max()))
    print("Min: " + str(df_joined['budget'].min()))

    print("## Runtime ##")
    print("Max: " + str(df_joined['runtime'].max()))
    print("Min: " + str(df_joined['runtime'].min()))

    print(df_joined['runtime'].sort_values(ascending=False).head(10))

    from sklearn import preprocessing
    scaler = preprocessing.MinMaxScaler()
    df_joined['budget_norm'] = scaler.fit_transform(df_joined[['budget']])
    df_joined['runtime_norm'] = scaler.fit_transform(df_joined[['runtime']])

    df_joined[['budget','runtime','budget_norm','runtime_norm']].head(10)

### 5.8. Remove features, which are not needed

In [38]:
features_to_remove = ['original_language','productionCompanies','productionCountries']
for i in features_to_remove:
    if i in df_joined.columns:
        df_joined = df_joined.drop(columns=i)
df_joined.head(3)

,budget,id,runtime,Documentary,Foreign,Action,Horror,War,Romance,Adventure,...,actor_John Wayne,actor_Donald Sutherland,actor_Robert De Niro,actor_Samuel L. Jackson,actor_Jackie Chan,actor_Michael Caine,actor_Christopher Lee,actor_Frank Welker,actor_John Carradine,actor_Gérard Depardieu
0,30000000,862,81.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,65000000,8844,104.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,0,15602,101.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


### 5.9. Remove Budget and Runtime with "0"
#### As we were not sure if this could have an impact on our prediciton results, we just used different datasets to compare both one with and one without records containing 0 in budget or runtime. 
#### After removing all entries without budget and runtime, the dataframe only contains around 9000 records

In [39]:
print("Budget Min: ",df_joined['budget'].min())
print("Runtime Min: ", df_joined['runtime'].min())
print("Length Dataset: ", len(df_joined))

df_without_null = df_joined[df_joined.budget != 0]
df_without_null = df_without_null[df_without_null.runtime != 0]

print("#####")
print("Budget Min After: ",df_without_null['budget'].min())
print("Runtime Min After: ", df_without_null['runtime'].min())
print("Length Dataset: ", len(df_without_null))

Budget Min:  0
Runtime Min:  0.0
Length Dataset:  43872
#####
Budget Min After:  1
Runtime Min After:  4.0
Length Dataset:  8705


## 6. Integrate Cluster ID
### This information is needed for the comparison - does the clusterid ('usercluster') help to create a better model?
#### rating_x : Rating average of the usercluster
#### rating_y : Overall rating average

In [40]:
df_cluster = pd.read_csv("clusteredratings.csv")
df_cluster.drop(columns=['timestamp'])
#df_cluster.head(5)

df_cluster = df_cluster.groupby(['usercluster','movieId'],as_index=False)['rating'].mean()
df_cluster.to_csv("cluster_tmp.csv", index=False)

df_cluster_merged = pd.merge(left=df_cluster,right=df_joined, left_on='movieId', right_on='movieId', how='inner')
#df_cluster[['movieId','usercluster','rating_x']].head(10)
df_cluster_merged.head(3)

,usercluster,movieId,rating_x,budget,id,runtime,Documentary,Foreign,Action,Horror,...,actor_John Wayne,actor_Donald Sutherland,actor_Robert De Niro,actor_Samuel L. Jackson,actor_Jackie Chan,actor_Michael Caine,actor_Christopher Lee,actor_Frank Welker,actor_John Carradine,actor_Gérard Depardieu
0,0,1,3.772727,30000000,862,81.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,3.937061,30000000,862,81.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,1,3.928131,30000000,862,81.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
df_cluster_without_zero = pd.merge(left=df_cluster,right=df_without_null, left_on='movieId', right_on='movieId', how='inner')
df_cluster_without_zero.head(3)

,usercluster,movieId,rating_x,budget,id,runtime,Documentary,Foreign,Action,Horror,...,actor_John Wayne,actor_Donald Sutherland,actor_Robert De Niro,actor_Samuel L. Jackson,actor_Jackie Chan,actor_Michael Caine,actor_Christopher Lee,actor_Frank Welker,actor_John Carradine,actor_Gérard Depardieu
0,0,1,3.772727,30000000,862,81.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,3.937061,30000000,862,81.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,1,3.928131,30000000,862,81.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


## 7. Export

### Export for Regression (general)

In [42]:
print("Length before export: " + str(len(df_joined)))
df_joined.to_csv("regressionPreprocessingGeneral.csv", index=False)

Length before export: 43872


### Export for Regression (containing cluster information)

In [43]:
print("Length before export: " + str(len(df_cluster_merged)))
df_cluster_merged.to_csv("regressionPreprocessingWithClustering.csv", index=False)

Length before export: 118856


### Export for Regression without zeros (general)

In [44]:
print("Length before export: " + str(len(df_without_null)))
df_without_null.to_csv("regressionPreprocessingWithoutZero.csv", index=False)

Length before export: 8705


### Export for Regression without zeros (containing cluster information)

In [45]:
print("Length before export: " + str(len(df_cluster_without_zero)))
df_cluster_without_zero.to_csv("regressionPreprocessingWithClusteringWithoutZero.csv", index=False)

Length before export: 34243


### Export for Classification (general)

In [46]:
#df_joined = pd.read_csv("regressionPreprocessing_backup.csv")

df_classification = df_joined.copy()
df_labels = pd.DataFrame(columns=['Rating_Label'])

for index, row in df_classification.iterrows():
    temp = pd.Series(['Rating_Label'])
    wert = row.rating
    if wert >= 4.5:
        temp['Rating_Label'] = 5
    elif wert >= 3.5:
        temp['Rating_Label'] = 4
    elif wert >= 2.5:
        temp['Rating_Label'] = 3
    elif wert >= 1.5: 
        temp['Rating_Label'] = 2
    elif wert >= 0.5: 
        temp['Rating_Label'] = 1
    else:
        temp['Rating_Label'] = 0
    df_labels = df_labels.append(temp, ignore_index=True)
df_classification['Rating_Label'] = df_labels['Rating_Label']
df_classification = df_classification.drop(columns=['rating'])
df_classification.head(5)

,budget,id,runtime,Documentary,Foreign,Action,Horror,War,Romance,Adventure,...,actor_Donald Sutherland,actor_Robert De Niro,actor_Samuel L. Jackson,actor_Jackie Chan,actor_Michael Caine,actor_Christopher Lee,actor_Frank Welker,actor_John Carradine,actor_Gérard Depardieu,Rating_Label
0,30000000,862,81.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,4
1,65000000,8844,104.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,3
2,0,15602,101.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,3
3,16000000,31357,127.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,3
4,0,11862,106.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,3


In [48]:
print("Length before export: " + str(len(df_classification)))
df_classification.to_csv("classificationPreprocessingGeneral.csv", index=False)

Length before export: 43872


### Export for Classification without zero (general)

In [49]:
df_classification = df_classification[df_classification.budget != 0]
df_classification = df_classification[df_classification.runtime != 0]
print("Length before export: " + str(len(df_classification)))
df_classification.to_csv("classificationPreprocessingGeneralWithoutZero.csv", index=False)

Length before export: 8705


### Export for Classification (containing cluster information)

In [50]:
df_classification = df_cluster_merged.copy()
df_labels = pd.DataFrame(columns=['Rating_Label'])

for index, row in df_classification.iterrows():
    temp = pd.Series(['Rating_Label'])
    wert = row.rating_x
    if wert >= 4.5:
        temp['Rating_Label'] = 5
    elif wert >= 3.5:
        temp['Rating_Label'] = 4
    elif wert >= 2.5:
        temp['Rating_Label'] = 3
    elif wert >= 1.5: 
        temp['Rating_Label'] = 2
    elif wert >= 0.5: 
        temp['Rating_Label'] = 1
    else:
        temp['Rating_Label'] = 0
    df_labels = df_labels.append(temp, ignore_index=True)
df_classification['Rating_Label'] = df_labels['Rating_Label']
df_classification = df_classification.drop(columns=['rating_x'])
df_classification.head(5)

,usercluster,movieId,budget,id,runtime,Documentary,Foreign,Action,Horror,War,...,actor_Donald Sutherland,actor_Robert De Niro,actor_Samuel L. Jackson,actor_Jackie Chan,actor_Michael Caine,actor_Christopher Lee,actor_Frank Welker,actor_John Carradine,actor_Gérard Depardieu,Rating_Label
0,0,1,30000000,862,81.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,4
1,1,1,30000000,862,81.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,4
2,2,1,30000000,862,81.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,4
3,3,1,30000000,862,81.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,4
4,4,1,30000000,862,81.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,4


In [51]:
print("Length before export: " + str(len(df_classification)))
df_classification.to_csv("classificationPreprocessingWithClustering.csv", index=False)

Length before export: 118856


### Export for Classification without zero (containing cluster information)

In [52]:
df_classification = df_classification[df_classification.budget != 0]
df_classification = df_classification[df_classification.runtime != 0]
print("Length before export: " + str(len(df_classification)))
df_classification.to_csv("classificationPreprocessingWithClusteringWithoutZero.csv", index=False)

Length before export: 34243
